# Module 3

This notebook performs the merging of survey question and response datasets with employee roster data and performs visualization for anaysis.

> [Merge Question and Response Data](#merge)

> [Score Employee by Measurement](#score)

> [Merge Employee and Roster Data](#roster)

> [Group by Department](#group)

> [Business Insights](#insight)

In [48]:
import pandas as pd 
import numpy as np 
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='plotly_mimetype+notebook_connected'

<a name="merge"></a>
## Merge Questions and Responses
Load the following datasets:
- responses_clean.csv download
- questions_clean.csv download

Merge the questions and responses Data Frames, using the question id as the join key so that each response matches up with the appropriate measurement in the measurement column.
Create a new column called “mean” and populate it with the following calculations for each employee:
- “Engagement”: the mean of questions 1 to 5
- “Leadership”: the mean of questions 6 to 8
- “Enablement”: the mean of questions 9 to 11
- “Alignment”: the mean of questions 12 to 14
- “Development”: the mean of questions 15 to 17

In [49]:
# Load clean data from CSV files
r_df = pd.read_csv('responses_clean.csv')
q_df = pd.read_csv('questions_clean.csv')

# Merge datasets
qr_df = r_df.merge(q_df, left_on='question_id', right_on='id').drop(columns='id')
qr_df.head(1)

,employee_id,question_id,answer,score,measurement,question
0,343,1,4,0.8,engagement,I am proud to work for [Company]


In [50]:
# Compute mean by category and employee
sum_df = qr_df.groupby(['employee_id','measurement'], as_index=False)['score'].mean().rename(columns={'score':'mean'})
sum_df.head()

,employee_id,measurement,mean
0,4,alignment,0.933333
1,4,development,0.933333
2,4,enablement,0.933333
3,4,engagement,0.800000
4,4,leadership,0.933333


<a name="score"></a>
## Score Each Employee by Measurement
Use the pivot_table function to transform the Dataframe you just created into a Dataframe with the following columns:
- Employee_id
  - Engagement: the mean of questions 1 to 5
  - Leadership: the mean of questions 6 to 8
  - Enablement: the mean of questions 9 to 11
  - Alignment: the mean of questions 12 to 14
  - Development: the mean of questions 15 to 17
- Add a column called “overall” and compute the overall satisfaction metric per employee. Note that this metric is the simple unweighted mean of all 5 “measurement” categories.
- Use plotly to produce a scatter matrix of Employee Survey Measures from the Pivot Table.

In [51]:
# List of measures
measures = sum_df['measurement'].unique()

# Pivot summarized employee data
pv_df = sum_df.pivot(index='employee_id',columns='measurement',values='mean')
pv_df['overall'] = pv_df.mean(axis=1)
pv_df.head()

measurement,alignment,development,enablement,engagement,leadership,overall
employee_id,,,,,,
4,0.933333,0.933333,0.933333,0.80,0.933333,0.906667
7,0.866667,0.933333,1.000000,0.92,0.933333,0.930667
10,0.933333,0.800000,0.933333,0.88,1.000000,0.909333
11,0.933333,0.866667,1.000000,0.84,1.000000,0.928000
14,0.866667,0.733333,1.000000,0.80,0.933333,0.866667


In [52]:
# Scatter matrix to review correlations
px.scatter_matrix(data_frame=pv_df, dimensions=measures, 
                title='Correlation Comparison of Survey Measures')

### Matrix Analysis
The variables that appear the most highly correlated based on the chart above are:
- Enablement and Engagement
- Leadership and Enablement

<a name="roster"></a>
## Merge Employee Scores with Roster Data:
Use the “employee_id” as the key upon which to merge the Data Frame you created with the roster_clean.csv file  download. Save this file as “roster_with_score.csv.”

In [53]:
# Load roster data and remove index column
e_df = pd.read_csv('roster_clean.csv').iloc[:,1:]

# Merge roster data onto summarized question & response dataset
mst_df = pv_df.merge(e_df, on='employee_id')
mst_df.head()

,employee_id,alignment,development,enablement,engagement,leadership,overall,title,last,first,...,function,department,location,age,sex,ethnicity,employment_status,tenure,count,tenure_id
0,4,0.933333,0.933333,0.933333,0.80,0.933333,0.906667,Senior Underwriter,Deleon,Josephine,...,Finance & Admin,Underwriting,"Mountain View, CA",25 - 34,F,asian,Full time,4 to less than 6 years,1,4
1,7,0.866667,0.933333,1.000000,0.92,0.933333,0.930667,Senior Underwriter,Bailey,Kristopher,...,Finance & Admin,Underwriting,"Mountain View, CA",25 - 34,M,asian,Full time,4 to less than 6 years,1,4
2,10,0.933333,0.800000,0.933333,0.88,1.000000,0.909333,Loan Officer,Scott,Iris,...,Loan Operations,East Coast Lending,"New York, NY",18 - 24,F,white,Full time,4 to less than 6 years,1,4
3,11,0.933333,0.866667,1.000000,0.84,1.000000,0.928000,VP Engineering,Xiong,Steven,...,CEO,Engineering,"Mountain View, CA",35 - 44,M,asian,Full time,4 to less than 6 years,1,4
4,14,0.866667,0.733333,1.000000,0.80,0.933333,0.866667,Engineer,Nguyen,Terri,...,Engineering,Back End,"Mountain View, CA",45 - 54,F,asian,Full time,4 to less than 6 years,1,4


In [61]:
# Save master datasets
mst_df.to_csv('roster_with_score.csv')

<a name="group"></a>
## Group by Department Names:
Use the merged dataset to compute the mean overall satisfaction by department. Perform the following tasks:
- Compute the mean overall satisfaction by department.
- Produce a Box plot to visualize the overall results with respect to each categorical variable of the roster: 
  - `categories = ['function', 'department','location', 'ethnicity','employment_status','tenure','sex']`
- Inspect the data and provide your observations as to which categorical variables are most likely to impact employee satisfaction.

In [55]:
# Mean satisfaction by department
dp_df = mst_df.groupby('department', as_index=False)['overall'].mean()
dp_df.head()

,department,overall
0,Advanced Technology,0.762667
1,Algorithm Development,0.790377
2,Back End,0.750175
3,Channel Marketing,0.739333
4,Compliance,0.783515


In [62]:
# Save department summary
dp_df.to_csv('responses_clean_with_dept.csv')

In [56]:
categories = ['function', 'department','location', 'ethnicity','employment_status','tenure','sex']
fig = plotly.subplots.make_subplots(rows=3,cols=3,specs=[[{'type':'box'} for x in range(0,3)] for y in range(0,3)])

for i, cat in enumerate(categories):
    fig.add_trace(go.Box(x=mst_df[cat], y=mst_df.overall), col=i % 3 + 1, row=i//3+1)
fig.update_layout(title='Reponse Distributions across Categories',height=700,width=1300,showlegend=False)
fig.show()

### Boxplot Analysis
Based on the plots generated, the variables that appear to contribute the most to differences in employee satisfaction are `department, ethnicity and tenure`. Employees working in specific departments and at the company over 4 years report higher satisfaction, while people of color report lower satisfaction.

<a name="insight"></a>
## Business Insights:
What business insights can you draw from looking at the data? Are there noticeable differences between categories? Be sure to support your observations with your data analysis. Which functional groups seem most satisfied? Which ones seem least satisfied?
- Are some locations more satisfied than others?
- Do you have reason to believe that tenure relates to overall satisfaction?
- Out of sex and ethnicity which seems most related to overall satisfaction?

### Insight Responses
- Satisfaction appears to be the highest in Mountain View and New York, while Atlanta and Chicago report notably lower satisfaction
- After 1 year, it does appear that each increased tenure category reports higher levels of job satisfaction than the previous group
- Ethnicity appears to have a more significant impact on satistfaction, with people of color reporting significantly lower satisfaction, while gender only shows small differences

In [63]:
!jupyter nbconvert --to html --template full LoanTronic_MasterData.ipynb

[NbConvertApp] Converting notebook LoanTronic_MasterData.ipynb to html
[NbConvertApp] Writing 444704 bytes to LoanTronic_MasterData.html
